In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from mlxtend.classifier import StackingCVClassifier
from sklearn import model_selection

In [3]:
df = pd.read_csv('NSL_new.csv')


In [4]:
df = df.drop( df[ (df.label != "normal") & (df.label != "neptune") & (df.label != "back") & (df.label != "land") & (df.label != "pod") & (df.label != "smurf") & (df.label != "teardrop") & (df.label != "teardrop") & (df.label != "mailbomb") & (df.label != "apache2") & (df.label != "processtable") & (df.label != "udpstorm") & (df.label != "worm")].index )
print(df.shape)

(113270, 123)


In [5]:
df = df.sample(n=20000)
df.head()


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH,label
110107,0,15876,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,normal
94699,0,221,2209,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,normal
44582,0,43,96,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,normal
12978,0,1032,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,smurf
73824,0,43,150,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,normal


In [6]:
X = df.drop(['label'], axis=1)
y = df['label']

In [7]:
 y= y.replace( to_replace =  'normal' , value = 0 )

y = y.replace( to_replace =  'neptune' , value = 1 )
y = y.replace( to_replace =  'back' , value = 1 )
y = y.replace( to_replace =  'land' , value = 1 )
y = y.replace( to_replace =  'pod' , value = 1 )
y = y.replace( to_replace =  'smurf' , value = 1 )
y = y.replace( to_replace =  'teardrop' , value = 1 )
y = y.replace( to_replace =  'mailbomb' , value = 1 )
y = y.replace( to_replace =  'apache2' , value = 1 )
y = y.replace( to_replace =  'processtable' , value = 1 )
y = y.replace( to_replace =  'udpstorm' , value = 1 )
y = y.replace( to_replace =  'worm' , value = 1 )

In [8]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier,StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from vecstack import stacking

In [9]:
from sklearn.metrics import accuracy_score , precision_score , recall_score , f1_score


scoring_metrics = { accuracy_score , precision_score , recall_score , f1_score }

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [11]:
from sklearn.preprocessing import StandardScaler

# Create Base Learners
sc = StandardScaler() 
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

from sklearn.decomposition import PCA

pca = PCA(n_components=6)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

In [12]:
from sklearn.tree import tree

# Create Base Learners
base_learners = [tree.DecisionTreeClassifier(max_depth= 13 , min_samples_split = 10 ) ,
                 AdaBoostClassifier(learning_rate=0.1 , n_estimators=1000)          
                ]
meta_learner = MLPClassifier(alpha=0.01 , hidden_layer_sizes=20 , learning_rate='constant' , solver='adam')

c:\users\ismail\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.tree.tree module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.tree. Anything that cannot be imported from sklearn.tree is now part of the private API.
  warnings.warn(message, FutureWarning)


In [13]:
# Initialize Stacking Classifier with the Meta Learner
S_train, S_test = stacking(base_learners,                   
                           X_train, y_train, X_test,   
                           regression=False, 
     
#                            mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric= accuracy_score , 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [2]

model  0:     [DecisionTreeClassifier]
    fold  0:  [0.99625000]
    fold  1:  [0.99525000]
    fold  2:  [0.99525000]
    fold  3:  [0.99500000]
    ----
    MEAN:     [0.99543750] + [0.00048007]
    FULL:     [0.99543750]

model  1:     [AdaBoostClassifier]
    fold  0:  [0.99500000]
    fold  1:  [0.99475000]
    fold  2:  [0.99425000]
    fold  3:  [0.99400000]
    ----
    MEAN:     [0.99450000] + [0.00039528]
    FULL:     [0.99450000]



In [14]:
model = meta_learner.fit(S_train, y_train)
y_pred = meta_learner.predict(S_test)
print('Accuracy score: [%.8f]' % accuracy_score(y_test, y_pred))
print('Precision score: [%.8f]' % precision_score(y_test, y_pred))
print('recall score: [%.8f]' % recall_score(y_test, y_pred))


Accuracy score: [0.99750000]
Precision score: [0.99746193]
recall score: [0.99619772]


In [15]:

stacking3 = { 'accuracy' : accuracy_score(y_test, y_pred) , 'Precision' : precision_score(y_test, y_pred) , 'recall' : recall_score(y_test, y_pred) }

stacking3 = pd.DataFrame(stacking3 , index=[0,1] )

In [16]:
stacking3.to_csv('NSL_DOS_Stacking_adaboost-DecisionTree-mlp.csv')
